In [2]:
# Import libraries

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
# Send an HTTP request and get the HTML content of the web page
url = 'http://159.203.110.236/obras/lista/?page=1&'
response = requests.get(url)
if response.status_code != 200:
    raise Exception("Failed to fetch the web page.")

In [5]:
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')


In [6]:
# Assuming the data is contained in a table with 'tr' and 'td' tags
table = soup.find('table')
data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append(cols)

In [7]:
# Convert the extracted data into a pandas DataFrame
df = pd.DataFrame(data)

In [9]:


# Set column names if available
df.columns = ['Id', 'Descricao', 'Tipo','Governo','Extensao(km)','Estagio','Contrato','Valor','Valor Medido','% Financeiro','%Fisico','Empresa','Inicio','Conclusao','Fonte de Recursos','Ver']

# Display the DataFrame
print(df)

      Id                                          Descricao          Tipo  \
0   None                                               None          None   
1      1  Acesso à Comunidade de Malhada da Roça, em São...  Pavimentação   
2      2             Acesso ao Distrito de Cepilho em Areia  Pavimentação   
3      3  Acesso ao Distrito de Pitombeira de Dentro em ...  Pavimentação   
4      4          Acesso ao Distrito de Reanscença, em Sapé  Pavimentação   
5      5  Acesso: Entr. PB-306 - Divisa PB/PE (Ligação p...  Pavimentação   
6      6           Caminho dos Engenhos (Areia-Alagoa Nova)  Pavimentação   
7      7       Entr. PB-354(Nova Olinda)/Pitombeira/Tavares  Pavimentação   
8      8                       Ligação: Jacaraú/Pedro Régis  Pavimentação   
9      9                              Nova Olinda - Tavares  Pavimentação   
10    10       PB-024: Entr. PB-018 / Mata da Chica (Conde)  Pavimentação   
11    11                              PB-025: BR-101/Lucena   Restauração   

In [14]:
# Drop empty column 
df = df.drop(columns='Ver')

In [15]:
#Drop NA
df = df.drop(index=0)

In [17]:
# repeat steps for other pages
def scrape_page(page_number):
    base_url = 'http://159.203.110.236/obras/lista/?page='
    url = f'{base_url}{page_number}&'
    
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch page {page_number}.")

    soup = BeautifulSoup(response.content, 'html.parser')

    # Assuming the data is contained in a table with 'tr' and 'td' tags
    table = soup.find('table')
    data = []
    for row in table.find_all('tr'):
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append(cols)

    return pd.DataFrame(data)

# Scrape pages 1, 2, 3, and 4 and concatenate the results into a single DataFrame
all_data_frames = [scrape_page(page_number) for page_number in range(1, 5)]
result_df = pd.concat(all_data_frames, ignore_index=True)

# Display the concatenated DataFrame
print(result_df)


      0                                                  1             2   \
0   None                                               None          None   
1      1  Acesso à Comunidade de Malhada da Roça, em São...  Pavimentação   
2      2             Acesso ao Distrito de Cepilho em Areia  Pavimentação   
3      3  Acesso ao Distrito de Pitombeira de Dentro em ...  Pavimentação   
4      4          Acesso ao Distrito de Reanscença, em Sapé  Pavimentação   
..   ...                                                ...           ...   
78  None                                               None          None   
79     1             PB-394: Entr. BR-230/Engenheiro Ávidos  Pavimentação   
80     2                  PB-395: Entr. PB-393/Santa Helena   Restauração   
81     3                    PB-400: Santa Inês/Divisa PB.PE  Pavimentação   
82     4                   Vicinal: Araruna - Pedra da Boca  Pavimentação   

              3      4            5            6                 7   \
0   

In [24]:
# Set column names
result_df.columns = ['Id', 'Descricao', 'Tipo','Governo','Extensao(km)','Estagio','Contrato','Valor','Valor Medido','% Financeiro','%Fisico','Empresa','Inicio','Conclusao','Fonte de Recursos','Ver']

In [25]:
# # Drop empty column 
result_df = result_df.drop(columns='Ver')


In [32]:
# Drop useless column
result_df = result_df.drop(columns='Id')

In [30]:
 #Drop NA
result_df = result_df.dropna(subset=['Id'])

In [33]:
result_df

,Descricao,Tipo,Governo,Extensao(km),Estagio,Contrato,Valor,Valor Medido,% Financeiro,%Fisico,Empresa,Inicio,Conclusao,Fonte de Recursos
1,"Acesso à Comunidade de Malhada da Roça, em São...",Pavimentação,João Azevedo,"2,90",Concluída,None,"R$ 3.079.159,50","R$ 3.079.159,50","100,00","100,00",MAC - Mesquita Andrade Construções LTDA.,Out 2020,Jun 2021,100 TESOURO DO ESTADO
2,Acesso ao Distrito de Cepilho em Areia,Pavimentação,João Azevedo,"3,00",Concluída,None,"R$ 2.930.628,42","R$ 2.930.628,42","100,00","100,00",Construtora Souza Reis LTDA,Abr 2021,Abr 2022,100 TESOURO DO ESTADO
3,Acesso ao Distrito de Pitombeira de Dentro em ...,Pavimentação,João Azevedo,"4,60",Em execução,None,"R$ 4.588.021,68","R$ 1.703.029,29","37,12","37,12",None,Set 2021,Dez 2022,100 TESOURO DO ESTADO
4,"Acesso ao Distrito de Reanscença, em Sapé",Pavimentação,João Azevedo,"3,50",Concluída,None,"R$ 2.776.925,32","R$ 2.776.925,32","100,00","100,00",MAC - Mesquita Andrade Construções LTDA.,Fev 2021,Fev 2022,100 TESOURO DO ESTADO
5,Acesso: Entr. PB-306 - Divisa PB/PE (Ligação p...,Pavimentação,João Azevedo,"1,20",Em projeto,None,R$ None,"R$ 1.500.000,00",,,None,,,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,PB-393 Cajazeiras/São João do Rio do Peixe,Restauração,João Azevedo,"23,00",Concluída,PJ-0014/2018,"R$ 2.577.185,30","R$ 2.577.185,30","100,00","100,00",Construtora Souza Reis LTDA,Ago 2019,Dez 2019,111 - CIDE
79,PB-394: Entr. BR-230/Engenheiro Ávidos,Pavimentação,João Azevedo,"14,10",Em execução,PJ-022/2020,"R$ 25.012.375,65","R$ 15.043.289,68","60,14","60,14",TAPAJÓS,Jun 2020,Jun 2023,100 TESOURO DO ESTADO
80,PB-395: Entr. PB-393/Santa Helena,Restauração,João Azevedo,"18,20",Concluída,None,"R$ 4.653.718,53","R$ 4.653.718,53","100,00","100,00",Construtora Souza Reis LTDA,Jul 2020,Jan 2021,100 TESOURO DO ESTADO
81,PB-400: Santa Inês/Divisa PB.PE,Pavimentação,João Azevedo,"10,20",Concluída,None,"R$ 12.460.793,83","R$ 12.460.793,86","100,00","100,00",COSAMPA Projetos e Construções LTDA.,Out 2021,Jun 2022,100 TESOURO DO ESTADO


In [ ]:
# Save as csv

In [34]:
result_df.to_csv('construcoes_pb.csv')